0. Load all data
1. Define numerics column
2. Load PCA and FA from pickle
3. Run PCA and FA over numerics
4. Append to existing DF
5. Cross-check columns in DF with selected ones
6. Remove extra columns
7. Split into train, test, validate
8. May need to run stratify for fold creation
9. Define early stopping func
10. Re-train on best best_n_tree_limit

In [2]:
from datetime import date, timedelta

from featureeng.times import get_trading_times_for_london_ny
from featureeng.features import *
import featureeng as feng
import data.readers as rd
import joblib


import matplotlib.pyplot as plt

import multiprocessing
import time
import threading
import json
import os

import pyarrow as pa
import pyarrow.parquet as pq

# Importing core libraries
import numpy as np
import pandas as pd
from time import time
import pprint
import joblib
from functools import partial

# Suppressing warnings because of skopt verbosity
import warnings
warnings.filterwarnings("ignore")

# Classifier/Regressor
from xgboost import XGBClassifier

# Feature selection
from BorutaShap import BorutaShap

# Data processing
from sklearn.preprocessing import OrdinalEncoder
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA, FactorAnalysis
from sklearn.mixture import GaussianMixture

import pickle
# Validation
from sklearn.model_selection import StratifiedKFold

#helper functions
from datetime import date, timedelta

from featureeng.times import get_trading_times_for_london_ny
from featureeng.features import *
import featureeng as feng
import data.readers as rd

import multiprocessing
import time
import threading
import json
import os

import pyarrow as pa
import pyarrow.parquet as pq

# Importing core libraries
import numpy as np
import pandas as pd
from time import time
import pprint
import joblib
from functools import partial

# Suppressing warnings because of skopt verbosity
import warnings
warnings.filterwarnings("ignore")


# Feature selection
from BorutaShap import BorutaShap

# Data processing
from sklearn.preprocessing import OrdinalEncoder
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA, FactorAnalysis
from sklearn.mixture import GaussianMixture

# Validation
from sklearn.model_selection import StratifiedKFold

root_dir = '/home/dcai/data/features'
instrument='EURCHF'
feature_set="3"



temp_save_dir="{}/{}".format(root_dir,instrument)

def save_df(save_dir, df, name):
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    pq.write_table(pa.Table.from_pandas(df), os.path.join(save_dir, name))


temp_save_dir="{}/{}".format(root_dir,instrument)
def read_parquet(save_dir, name):
    return pq.ParquetFile(os.path.join(save_dir, name)).read().to_pandas()



def list_of_files(root_dir):
    filenames = []
    for root, dirs, files in os.walk(root_dir):
        for file in files:
            # Check if the file is a CSV file
            if file.endswith('.parquet'):
                filenames.append(os.path.join(root, file))
    return filenames

def read_in_files(file_list):
    dfs = []
    for f in file_list:
        dfs.append(pq.ParquetFile(f).read().to_pandas())
    return pd.concat(dfs).sort_index()

def train_valid_test(instrument, feature_set, root_dir='/home/dcai/data/features',  train_size=0.75):
    files = list_of_files("{}/{}/{}".format(root_dir,instrument, feature_set))
    files.sort()

    train_index = int(len(files)*train_size)
    train = files[0:train_index]
    valid_test = files[train_index:]


    valid = valid_test[0::2]
    test = valid_test[1::2]

    return {'train': read_in_files(train),
            'valid': read_in_files(valid),
            'test': read_in_files(test)
           }

def extract_features_from_gmm_cols(gmm_cols):
    result = [splits[0] for string in gmm_cols for splits in [string.split("_gmm_")] if len(splits) > 1]
    return list(set(result))


def save_df(save_dir, df, name):
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    pq.write_table(pa.Table.from_pandas(df), os.path.join(save_dir, name))



def read_parquet(save_dir, name):
    return pq.ParquetFile(os.path.join(save_dir, name)).read().to_pandas()



    


#allow logloss and classification error plots for each iteraetion of xgb model
def plot_compare(metrics,eval_results,epochs):
    for m in metrics:
        test_score = eval_results['val'][m]
        train_score = eval_results['train'][m]
        rang = range(0, epochs)
        plt.rcParams["figure.figsize"] = [6,6]
        plt.plot(rang, test_score,"c", label="Val")
        plt.plot(rang, train_score,"orange", label="Train")
        title_name = m + " plot"
        plt.title(title_name)
        plt.xlabel('Iterations')
        plt.ylabel(m)
        lgd = plt.legend()
        plt.show()    
    
#pre_selected_cols = ['MeanPrice_x', 'close_MIN_1D_5', 'close_MIN_1D_20', 'close_MIN_1D_60', 'close_MAX_1D_5', 'close_MAX_1D_20', 'close_MAX_1D_60', 'close_AVG_1D_5', 'close_AVG_1D_20', 'close_AVG_1D_60', 'close_STDEV_1D_5', 'close_STDEV_1D_20', 'close_STDEV_1D_60', 'close_MEDIAN_1D_5', 'close_MEDIAN_1D_20', 'close_MEDIAN_1D_60', 'close_MIN_1H_5', 'close_MIN_1H_20', 'close_MIN_1H_60', 'close_MAX_1H_5', 'close_MAX_1H_20', 'close_MAX_1H_60', 'close_AVG_1H_5', 'close_AVG_1H_20', 'close_AVG_1H_60', 'close_STDEV_1H_5', 'close_STDEV_1H_20', 'close_STDEV_1H_60', 'close_MEDIAN_1H_5', 'close_MEDIAN_1H_20', 'close_MEDIAN_1H_60', 'London', 'NY', 'rolling_1D_60_3_1', 'rolling_1D_60_3_2', 'rolling_1D_60_3_3', 'ewm_1D_60_3_1', 'ewm_1D_60_3_2', 'ewm_1D_60_3_3', 'rolling_1D_20_4_1', 'rolling_1D_20_4_2', 'rolling_1D_20_4_3', 'rolling_1D_20_4_4', 'ewm_1D_20_4_1', 'ewm_1D_20_4_2', 'ewm_1D_20_4_3', 'ewm_1D_20_4_4', 'rolling_4H_20_4_1', 'rolling_4H_20_4_2', 'rolling_4H_20_4_3', 'rolling_4H_20_4_4', 'ewm_4H_20_4_1', 'ewm_4H_20_4_2', 'ewm_4H_20_4_3', 'ewm_4H_20_4_4', 'rolling_1D_60_20_4_1', 'rolling_1D_60_20_4_2', 'rolling_1D_60_20_4_3', 'rolling_1D_60_20_4_4', 'rolling_4H_60_20_4_1', 'rolling_4H_60_20_4_2', 'rolling_4H_60_20_4_3', 'rolling_4H_60_20_4_4', 'MeanPrice_y', 'EUR', 'CHF', 'USD', 'GBP', 'fa_0', 'fa_1', 'fa_2', 'fa_3', 'fa_4', 'fa_5', 'fa_6', 'fa_7', 'fa_8', 'fa_9', 'fa_10', 'fa_11', 'fa_12', 'fa_13', 'fa_14', 'fa_15', 'fa_16', 'fa_17', 'fa_18', 'fa_19', 'fa_20', 'fa_21', 'fa_22', 'fa_23', 'fa_24', 'fa_25', 'fa_26', 'fa_27', 'fa_28', 'fa_29', 'fa_30', 'fa_31', 'fa_32', 'fa_33', 'fa_34', 'fa_35', 'fa_36', 'fa_37', 'fa_38', 'fa_39', 'fa_40', 'fa_41', 'fa_42', 'fa_43', 'fa_44', 'fa_45', 'fa_46', 'fa_47', 'fa_48', 'fa_49', 'fa_50', 'fa_51', 'fa_52', 'fa_53', 'fa_54', 'fa_55', 'fa_56', 'fa_57', 'fa_58', 'fa_59', 'fa_60', 'fa_61', 'fa_62', 'fa_63', 'pca_0', 'pca_1', 'pca_2', 'pca_3', 'pca_4', 'pca_5', 'pca_6', 'pca_7', 'pca_8', 'pca_9', 'pca_10', 'pca_11', 'pca_12', 'pca_13', 'pca_14', 'pca_15', 'pca_16', 'pca_17', 'pca_18', 'pca_19', 'pca_20', 'pca_21', 'pca_22', 'pca_23', 'pca_24', 'pca_25', 'pca_26', 'pca_27', 'pca_28', 'pca_29', 'pca_30', 'pca_31', 'pca_32', 'pca_33', 'pca_34', 'pca_35', 'pca_36', 'pca_37', 'pca_38', 'pca_39', 'pca_40', 'pca_41', 'pca_42', 'pca_43', 'pca_44', 'pca_45', 'pca_46', 'pca_47', 'pca_48', 'pca_49', 'pca_50', 'pca_51', 'pca_52', 'pca_53', 'pca_54', 'pca_55', 'pca_56', 'pca_57', 'pca_58', 'pca_59', 'pca_60', 'pca_61', 'pca_62', 'pca_63']
#X_train = read_parquet(temp_save_dir, 'X_train.parquet')#[pre_selected_cols]
#Y_train = read_parquet(temp_save_dir, 'Y_train.parquet')

In [13]:
start_time = time()    
result = train_valid_test(instrument=instrument, feature_set=feature_set)
end_time = time()
print(f'Execution time: {end_time - start_time:.3f} seconds')

Execution time: 11.863 seconds


In [7]:

numerics = ['MeanPrice', 'close_MIN_1D_5', 'close_MIN_1D_20', 'close_MIN_1D_60',
       'close_MAX_1D_5', 'close_MAX_1D_20', 'close_MAX_1D_60',
       'close_AVG_1D_5', 'close_AVG_1D_20', 'close_AVG_1D_60',
       'close_STDEV_1D_5', 'close_STDEV_1D_20', 'close_STDEV_1D_60',
       'close_MEDIAN_1D_5', 'close_MEDIAN_1D_20', 'close_MEDIAN_1D_60',
       'close_MIN_1H_5', 'close_MIN_1H_20', 'close_MIN_1H_60',
       'close_MAX_1H_5', 'close_MAX_1H_20', 'close_MAX_1H_60',
       'close_AVG_1H_5', 'close_AVG_1H_20', 'close_AVG_1H_60',
       'close_STDEV_1H_5', 'close_STDEV_1H_20', 'close_STDEV_1H_60',
       'close_MEDIAN_1H_5', 'close_MEDIAN_1H_20', 'close_MEDIAN_1H_60',
       'rolling_1D_60_3_1', 'rolling_1D_60_3_2',
       'rolling_1D_60_3_3', 'ewm_1D_60_3_1', 'ewm_1D_60_3_2', 'ewm_1D_60_3_3',
       'rolling_1D_20_4_1', 'rolling_1D_20_4_2', 'rolling_1D_20_4_3',
       'rolling_1D_20_4_4', 'ewm_1D_20_4_1', 'ewm_1D_20_4_2', 'ewm_1D_20_4_3',
       'ewm_1D_20_4_4', 'rolling_4H_20_4_1', 'rolling_4H_20_4_2',
       'rolling_4H_20_4_3', 'rolling_4H_20_4_4', 'ewm_4H_20_4_1',
       'ewm_4H_20_4_2', 'ewm_4H_20_4_3', 'ewm_4H_20_4_4',
       'rolling_1D_60_20_4_1', 'rolling_1D_60_20_4_2', 'rolling_1D_60_20_4_3',
       'rolling_1D_60_20_4_4', 'rolling_4H_60_20_4_1', 'rolling_4H_60_20_4_2',
       'rolling_4H_60_20_4_3', 'rolling_4H_60_20_4_4']

labels = [ 'label_60_5_buy', 'label_60_5_sell', 'label_60_5_stay']


In [ ]:
train = result['train']
test = result['test']
valid = result['valid']

X_train = train[numerics]
Y_train = train[labels]

X_test = test[numerics]
Y_test = test[labels]

X_valid = valid[numerics]
Y_valid = valid[labels]

result = None
train = None
test = None
valid = None

In [24]:


print(os.path.join(temp_save_dir, 'fa.pickle'))

with open(os.path.join(temp_save_dir, 'fa.pickle'), 'rb') as f:
    fa = pickle.load(f)
    
with open(os.path.join(temp_save_dir, 'pca.pickle'), 'rb') as f:
    pca = pickle.load(f)
    
with open(os.path.join(temp_save_dir, 'km.pickle'), 'rb') as f:
    km = pickle.load(f)
    
with open(os.path.join(temp_save_dir, 'y_stratified.pickle'), 'rb') as f:
    y_stratified = pickle.load(f)

with open(os.path.join(temp_save_dir, 'selected_feats-5.pickle'), 'rb') as f:
    selected_feats = pickle.load(f)



/home/dcai/data/features/EURCHF/fa.pickle


In [16]:
fa_feats = [f'fa_{i}'for i in range(len(numerics))]#[:4]

X_train[fa_feats] = fa.transform(X_train[numerics])
X_test[fa_feats] = fa.transform(X_test[numerics])
X_valid[fa_feats] = fa.transform(X_valid[numerics])

In [18]:
pca_feats = [f'pca_{i}'for i in range(len(numerics))]

In [ ]:
pca_feats_raw = pca.transform(X_train[numerics])

In [23]:
X_train[numerics].shape

(13080771, 61)

In [25]:
X_train[pca_feats] = pca.transform(X_train[numerics])
X_test[pca_feats] = pca.transform(X_test[numerics])
X_valid[pca_feats] = pca.transform(X_valid[numerics])
#valid[pca_feats] = pca.transform(valid[numerics])

In [27]:
X_train.columns.values

array(['MeanPrice', 'close_MIN_1D_5', 'close_MIN_1D_20',
       'close_MIN_1D_60', 'close_MAX_1D_5', 'close_MAX_1D_20',
       'close_MAX_1D_60', 'close_AVG_1D_5', 'close_AVG_1D_20',
       'close_AVG_1D_60', 'close_STDEV_1D_5', 'close_STDEV_1D_20',
       'close_STDEV_1D_60', 'close_MEDIAN_1D_5', 'close_MEDIAN_1D_20',
       'close_MEDIAN_1D_60', 'close_MIN_1H_5', 'close_MIN_1H_20',
       'close_MIN_1H_60', 'close_MAX_1H_5', 'close_MAX_1H_20',
       'close_MAX_1H_60', 'close_AVG_1H_5', 'close_AVG_1H_20',
       'close_AVG_1H_60', 'close_STDEV_1H_5', 'close_STDEV_1H_20',
       'close_STDEV_1H_60', 'close_MEDIAN_1H_5', 'close_MEDIAN_1H_20',
       'close_MEDIAN_1H_60', 'rolling_1D_60_3_1', 'rolling_1D_60_3_2',
       'rolling_1D_60_3_3', 'ewm_1D_60_3_1', 'ewm_1D_60_3_2',
       'ewm_1D_60_3_3', 'rolling_1D_20_4_1', 'rolling_1D_20_4_2',
       'rolling_1D_20_4_3', 'rolling_1D_20_4_4', 'ewm_1D_20_4_1',
       'ewm_1D_20_4_2', 'ewm_1D_20_4_3', 'ewm_1D_20_4_4',
       'rolling_4H_20_4_1'

In [30]:
any_missing = list(set(selected_feats)-set(X_train.columns.values))
print(any_missing)
print("-----")
selected_feats_3 = list(set(selected_feats)-set(any_missing))

selected_feats_3.sort()

#selected_feats_3 = [feat for feat in selected_feats_3 if not "gmm" in feat]
print(selected_feats_3)

print("-----")

[]
-----
['MeanPrice', 'close_AVG_1D_20', 'close_AVG_1D_5', 'close_AVG_1D_60', 'close_AVG_1H_20', 'close_AVG_1H_5', 'close_AVG_1H_60', 'close_MAX_1D_20', 'close_MAX_1D_5', 'close_MAX_1D_60', 'close_MAX_1H_20', 'close_MAX_1H_5', 'close_MAX_1H_60', 'close_MEDIAN_1D_20', 'close_MEDIAN_1D_5', 'close_MEDIAN_1D_60', 'close_MEDIAN_1H_20', 'close_MEDIAN_1H_5', 'close_MEDIAN_1H_60', 'close_MIN_1D_20', 'close_MIN_1D_5', 'close_MIN_1D_60', 'close_MIN_1H_20', 'close_MIN_1H_5', 'close_MIN_1H_60', 'close_STDEV_1D_20', 'close_STDEV_1D_5', 'close_STDEV_1D_60', 'close_STDEV_1H_20', 'close_STDEV_1H_5', 'close_STDEV_1H_60', 'ewm_1D_20_4_1', 'ewm_1D_20_4_2', 'ewm_1D_20_4_3', 'ewm_1D_20_4_4', 'ewm_1D_60_3_1', 'ewm_1D_60_3_2', 'ewm_1D_60_3_3', 'ewm_4H_20_4_1', 'ewm_4H_20_4_2', 'ewm_4H_20_4_3', 'ewm_4H_20_4_4', 'fa_0', 'fa_1', 'fa_10', 'fa_11', 'fa_12', 'fa_2', 'fa_3', 'fa_4', 'fa_5', 'fa_6', 'fa_7', 'fa_8', 'fa_9', 'pca_0', 'pca_1', 'pca_10', 'pca_11', 'pca_12', 'pca_13', 'pca_14', 'pca_15', 'pca_16', 'pca_

In [31]:
trainX = X_train[selected_feats_3]
trainY = Y_train[labels].values.argmax(axis=1)

validateX = X_valid[selected_feats_3]
validateY = Y_valid[labels].values.argmax(axis=1)



testX = X_test[selected_feats_3]
testY = Y_test[labels].values.argmax(axis=1)


print(trainX.shape)
print(trainY.shape)

print(validateX.shape)
print(validateY.shape)

print(testX.shape)
print(testY.shape)

(13080771, 134)
(13080771,)
(2168466, 134)
(2168466,)
(2114119, 134)
(2114119,)


In [37]:
save_df(temp_save_dir, trainX, "trainX.parqet")

save_df(temp_save_dir, validateX, "validateX.parqet")

save_df(temp_save_dir, testX, "testX.parqet")


with open(os.path.join(temp_save_dir, 'trainY.pickle'), 'wb') as f:
    pickle.dump(trainY, f)
    
with open(os.path.join(temp_save_dir, 'validateY.pickle'), 'wb') as f:
    pickle.dump(validateY, f)
    
with open(os.path.join(temp_save_dir, 'testY.pickle'), 'wb') as f:
    pickle.dump(testY, f)

In [33]:
from xgboost import XGBRegressor, XGBClassifier, DMatrix

dtrain = DMatrix(trainX, trainY, enable_categorical=False)
dvalid =  DMatrix(validateX, label=validateY, enable_categorical=False)
dtest = DMatrix(testX, label=testY, enable_categorical=False)

In [34]:

from collections import OrderedDict



## parameters to be tuned
tune_dic = OrderedDict()

tune_dic['max_depth']= [5,6,7,8,10,15,20,25] ## maximum tree depth
tune_dic['subsample']=[0.5,0.6,0.7,0.8,0.9,1.0] ## proportion of training instances used in trees
tune_dic['colsample_bytree']= [0.5,0.6,0.7,0.8,0.9,1.0] ## subsample ratio of columns
tune_dic['eta']= [0.01,0.25,0.05,0.10,]  ## learning rate
tune_dic['gamma']= [0.00,0.05,0.10,0.15,0.20]  ## minimum loss function reduction required for a split
#tune_dic['scale_pos_weight']=[30,40,50,300,400,500,600,700] ## relative weight of positive/negative instances

lengths = [len(lst) for lst in tune_dic.values()]

combs=1
for i in range(len(lengths)):
    combs *= lengths[i]
print('Total number of combinations: {:16d}'.format(combs))  

maxiter=500

columns=[*tune_dic.keys()]+['F-Score','Best F-Score']
results = pd.DataFrame(index=range(maxiter), columns=columns) ## dataframe to hold training results


Total number of combinations:             5760


In [35]:
def next_choice(cur_params=None):
    ## returns a random combination of the variable parameters (if cur_params=None)
    ## or a random neighboring combination from cur_params
    if cur_params:
        ## chose parameter to change
        ## parameter name and current value
        choose_param_name, cur_value = random.choice(list(cur_choice.items())) ## parameter name 
       
        all_values =  list(tune_dic[choose_param_name]) ## all values of selected parameter
        cur_index = all_values.index(cur_value) ## current index of selected parameter
        
        if cur_index==0: ## if it is the first in the range select the second one
            next_index=1
        elif cur_index==len(all_values)-1: ## if it is the last in the range select the previous one
            next_index=len(all_values)-2
        else: ## otherwise select the left or right value randomly
            direction=np.random.choice([-1,1])
            next_index=cur_index + direction

        next_params = dict((k,v) for k,v in cur_params.items())
        next_params[choose_param_name] = all_values[next_index] ## change the value of the selected parameter
        print('selected move: {:10s}: from {:6.2f} to {:6.2f}'.
              format(choose_param_name, cur_value, all_values[next_index] ))
    else: ## generate a random combination of parameters
        next_params=dict()
        for i in range(len(tune_dic)):
            key = [*tune_dic.keys()][i] 
            values = [*tune_dic.values()][i]
            next_params[key] = np.random.choice(values)
    return(next_params)